# Projeto 2

**1 -** Importando as bibliotecas que serão usadas

In [ ]:
import moviepy.editor as mpe
import numpy as np

**2 -** Abrindo o vídeo
 e salvando em uma variável

In [ ]:
videoprojeto2 = mpe.VideoFileClip("video_projeto2.mp4")

In [ ]:
print('Tamanho:', videoprojeto2.size)
print('FPS:', videoprojeto2.fps)
print('Duração:', videoprojeto2.duration, 'segundos')
print('Número de frames:', videoprojeto2.reader.nframes)

**3 -** Invertendo a imagem no eixo a cada 20 segundos

In [ ]:
def inverter_eixo(get_frame, t):
    frame = get_frame(t)
    if (int(t) // 20) % 2 == 1:
        return frame[:, ::-1]
    return frame

videoprojeto2_invertido = videoprojeto2.fl(inverter_eixo)

videoprojeto2_invertido.ipython_display(width=480, maxduration=132)

**4 -** Diminuindo o som, de forma gradativa, a cada 30 segundos, de forma que os últimos 10 segundos fique em silêncio 

In [ ]:
def reduce_volume(t, duration):
    intervalos = [0, 30, 60, 90, 120, duration - 10]  
    volumes = [1.0, 0.8, 0.6, 0.4, 0.2, 0.0]  

    if np.isscalar(t): 
        for i in range(len(intervalos)):
            if t < intervalos[i]:
                return volumes[i]
        return volumes[-1]
    else:
        t = np.array(t)
        out = np.zeros_like(t, dtype=float)
        for i in range(len(intervalos)):
            out = np.where((t >= intervalos[i - 1] if i > 0 else t >= 0) & (t < intervalos[i]), volumes[i], out)
        return np.where(t >= intervalos[-1], volumes[-1], out)



videoprojeto2_audio = videoprojeto2_invertido.audio
videoprojeto2_audio_reduzido = videoprojeto2_invertido.audio.fl(
    lambda gf, t: gf(t) * reduce_volume(t, videoprojeto2_invertido.duration)[:, np.newaxis]
)

videoprojeto2_diminuido = videoprojeto2_invertido.set_audio(videoprojeto2_audio_reduzido)

videoprojeto2_diminuido.ipython_display(width=480, maxduration=132)

**5 -** Após o primeiro minuto, 60 segundos, fazer um corte para o segundo 80

In [ ]:
videoprojeto2_cortado_clip1 = videoprojeto2_diminuido.subclip(0, 60)
videoprojeto2_cortado_clip2 = videoprojeto2_diminuido.subclip(80, videoprojeto2_diminuido.duration)

videoprojeto2_cortado = mpe.concatenate_videoclips([videoprojeto2_cortado_clip1, videoprojeto2_cortado_clip2])

videoprojeto2_cortado.ipython_display(width=480, maxduration=132)

**6 -** Por fim, insira os 20 segundos cortados acima no fim do video

In [ ]:
videoprojeto2_cortado_clip = videoprojeto2_diminuido.subclip(60, 80)
videoprojeto2_final = mpe.concatenate_videoclips([videoprojeto2_cortado, videoprojeto2_cortado_clip])

videoprojeto2_final.write_videofile("videoprojeto2_final.mp4")
